In [3]:
!pip3 install --upgrade git+https://github.com/twintproject/twint.git@origin/master#egg=twint
import pandas as pd
import twint
import seaborn as sns
import matplotlib.pyplot as plt

  Cloning https://github.com/twintproject/twint.git (to revision origin/master) to /tmp/pip-install-w1kc0opt/twint_a9b0d75fc8dd4e9495f5ad250ac0ed77
  Running command git clone --filter=blob:none --quiet https://github.com/twintproject/twint.git /tmp/pip-install-w1kc0opt/twint_a9b0d75fc8dd4e9495f5ad250ac0ed77
  Running command git checkout -q origin/master
  Resolved https://github.com/twintproject/twint.git to commit origin/master
  Preparing metadata (setup.py) ... done
  Using cached aiohttp-3.8.1-cp38-cp38-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (1.3 MB)
  Using cached aiodns-3.0.0-py3-none-any.whl (5.0 kB)
  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached cchardet-2.1.7-cp38-cp38-manylinux2010_x86_64.whl (265 kB)
  Using cached dataclasses-0.6-py3-none-any.whl (14 kB)
  Using cached elasticsearch-8.3.3-py3-none-any.whl (382 kB)
  Using cached PySocks-1.7.1-py3-none-any.whl (16 kB)
  Using cached pandas-1.4.3-

In [5]:
!pip install nest_asyncio
!pip install tqdm

  Using cached nest_asyncio-1.5.5-py3-none-any.whl (5.2 kB)
  Using cached tqdm-4.64.0-py2.py3-none-any.whl (78 kB)


In [43]:
try:#check if the file exist so you dont have to scrap from begining
    df = pd.read_csv('tweets.csv') 
    df = df.drop(df.columns[0], axis=1)
    df = df.dropna(subset='date')
    for date in df.date.unique():
        if len(date.split('-')) != 3:
            indexes = df[df.date==date].index
            for index in indexes:
                df = df.drop(index)
    df['date'] = pd.to_datetime(df['date'])
    date = max(df['date'])
    date = str(date).split(' ')[0] #get the date in the string format for twint
except:
    print('No tweets file')
df.shape

(611702, 37)

In [44]:
date

'2022-08-22'

In [45]:
import nest_asyncio
nest_asyncio.apply()

In [46]:
c = twint.Config()
c.Search = ['Formula 1']
c.Limit = 200000
c.Since = date
c.Store_csv = True
c.Output = 'new_tweets.csv'
c.Hide_output = True
twint.run.Search(c)

[!] No more data! Scraping will stop now.
found 0 deleted tweets in this search.


In [47]:
df1 = pd.read_csv('new_tweets.csv')
df1['date'] = pd.to_datetime(df1['date'])
date = max(df1['date'])
date = str(date).split(' ')[0] 
df1['tweet']
date

'2022-08-23'

In [48]:
tweets = pd.concat([df, df1]) #mergin both df so you have only one
tweets = tweets.drop_duplicates()
tweets.to_csv('tweets.csv', index=False)
tweets.shape

(769404, 37)

**Filter the new tweets from the old ones to not double translate**

In [6]:
df_t = pd.read_csv('tweets_translated.csv')
df_t['time_stamp'] = df_t['date'] + ' ' + df_t['time']
df_t['time_stamp'] = pd.to_datetime(df_t['time_stamp'])
date = max(df_t['time_stamp'])
date
df_t.shape

(368842, 41)

In [7]:
df1 = pd.read_csv('new_tweets.csv')
df1['time_stamp'] = df1['date'] + ' ' + df1['time']
df1['time_stamp'] = pd.to_datetime(df1['time_stamp'])
new = df1[df1['time_stamp'] > date]
new.shape

(10776, 37)

In [8]:
!pip install deep_translator

  Using cached beautifulsoup4-4.11.1-py3-none-any.whl (128 kB)
  Using cached requests-2.28.1-py3-none-any.whl (62 kB)
  Using cached soupsieve-2.3.2.post1-py3-none-any.whl (37 kB)
  Using cached urllib3-1.26.12-py2.py3-none-any.whl (140 kB)
  Using cached certifi-2022.6.15-py3-none-any.whl (160 kB)
  Using cached charset_normalizer-2.1.1-py3-none-any.whl (39 kB)
  Using cached idna-3.3-py3-none-any.whl (61 kB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
redshift-connector 2.0.900 requires pytz<2021.9,>=2020.1, but you have pytz 2022.2.1 which is incompatible.
redshift-connector 2.0.900 requires requests<2.26.1,>=2.23.0, but you have requests 2.28.1 which is incompatible.


In [9]:
# Translate tweets for better analysis
from tqdm import tqdm
from deep_translator import GoogleTranslator
tqdm.pandas()
new['tweet'] = new['tweet'].progress_apply(lambda tweet: GoogleTranslator(source='auto', target='en').translate(tweet))
new['tweet']

100%|██████████| 10776/10776 [49:13<00:00,  3.65it/s] 


576534                 @MrLH44 @F1  https://t.co/gPtJACIYBA
576535    @vettelfan_niki @F1 @SchumacherMick 🥲  https:/...
576536    F1 quiz: Every driver to score a point in the ...
576537    Former Formula 1 chief Ecclestone denies fraud...
576538     @Lokihan3 @KazuneKZ_ @F1 https://t.co/jfBglZXlUn
                                ...                        
587305               @F1Manager @F1 Grab this ratio instead
587306    # F1 | @Anto_Giovinazzi with #Haas in FP1 at M...
587307                          @F1Manager @F1 #f1Manager 🔋
587308                         @F1Manager @F1 #F1Manager Ⓜ️
587309                     @F1 😁😁😁😁 https://t.co/6wHG24FObs
Name: tweet, Length: 10776, dtype: object

In [10]:
tweets = pd.concat([new, df_t]) #mergin both df so you have only one
tweets = tweets.drop_duplicates()
tweets['tweet']

576534                 @MrLH44 @F1  https://t.co/gPtJACIYBA
576535    @vettelfan_niki @F1 @SchumacherMick 🥲  https:/...
576536    F1 quiz: Every driver to score a point in the ...
576537    Former Formula 1 chief Ecclestone denies fraud...
576538     @Lokihan3 @KazuneKZ_ @F1 https://t.co/jfBglZXlUn
                                ...                        
368837    1968: Born Ricardo Rosset, Brazilian Formula 1...
368838    Shitty no one knows how to race with formula 1...
368839    ((The Most Read)) #Noticias #mundotuerca #form...
368840    💖🐶 'Fabbio' brings all the attitude of formula...
368841    @marca @Carlossainz55 RIDICULOUS .... that's h...
Name: tweet, Length: 379618, dtype: object

In [11]:
tweets = pd.concat([new, df_t]) #mergin both df so you have only one
tweets = tweets.drop_duplicates()
tweets.to_csv('tweets_translated.csv', index=False)

In [12]:
data = pd.read_csv('tweets_translated.csv')
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379618 entries, 0 to 379617
Data columns (total 41 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   id               379618 non-null  object 
 1   conversation_id  307741 non-null  float64
 2   created_at       307741 non-null  object 
 3   date             379616 non-null  object 
 4   time             379616 non-null  object 
 5   timezone         307741 non-null  float64
 6   user_id          379616 non-null  float64
 7   username         379616 non-null  object 
 8   name             379610 non-null  object 
 9   place            91 non-null      object 
 10  tweet            379614 non-null  object 
 11  language         368240 non-null  object 
 12  mentions         368240 non-null  object 
 13  urls             368240 non-null  object 
 14  photos           368240 non-null  object 
 15  replies_count    368240 non-null  float64
 16  retweets_count   368240 non-null  floa